In [ ]:
# STARTUP
from keys import api, secret
from binance.um_futures import UMFutures
import pandas as pd
import numpy as np
import ta
from backtesting import Backtest, Strategy
from binance_futures import get_tickers_usdt, klines_datetime, klines
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
import concurrent.futures
import time

warnings.filterwarnings("ignore")
np.seterr(divide='ignore')

client = UMFutures(key=api, secret=secret)

In [2]:
# STRATEGY
ROC_PERIOD = 12 # x 5m = 1h
VOL_PERIOD = 48 # x 5m = 4h
ROC_TH = 1.8
VOL_TH = 4

def roc(df, period=1):
    return ta.momentum.ROCIndicator(pd.Series(df), window=period).roc()

class str_GOODSELL(Strategy):
    tp_opt = 0.032
    sl_opt = 0.018

    def init(self):
        self.roc = self.I(roc, self.data.Close, ROC_PERIOD)

    def next(self):
        close = self.data.Close[-1]

        n = VOL_PERIOD
        volume_last_n = pd.Series(self.data.Volume[-(n+1):-1])
        th = volume_last_n.mean() * VOL_TH
        if len(volume_last_n) < n:
            return
        
        if (not self.position and
            (self.roc[-1] < -ROC_TH)
            and self.data.Volume[-1] > th
        ):
            self.sell(size=0.01, tp=(1-self.tp_opt)*close, sl=(1+self.sl_opt)*close)


In [ ]:
# BACKTEST
TP = 0.032
SL = 0.018
VOLUME = 10
FEES = 0.0007

SYMBOLS = get_tickers_usdt()

DELTA = 0
DAYS = 30
DT_END_TEST = datetime.now() - timedelta(days=DELTA)


def run_backtest(symbol):
    try:
        kl = klines_datetime(symbol, '5m', DAYS, DT_END_TEST)
        bt = Backtest(kl, str_GOODSELL, cash=100, margin=1/10, commission=FEES, trade_on_close=False)
        stats = bt.run(tp_opt=TP, sl_opt=SL)
        stats['symbol'] = symbol

        time.sleep(0.5)

        return stats
    
    except Exception as err:
        print(f"Error - {symbol}: {err}")
        return None

data = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(run_backtest, SYMBOLS), total=len(SYMBOLS)))


# RESULTS
df = pd.DataFrame([stats for stats in results if stats is not None])
df = df.set_index('symbol')
df = df.sort_values('SQN', ascending=False)

df['win'] = (df['Win Rate [%]'] * df['# Trades']) / 100
win = df['win'].sum()
trades = df['# Trades'].sum()
profit = (win*TP*VOLUME) - ((trades-win)*SL*VOLUME) - (trades*VOLUME*FEES)

print(f"# Trades: {trades} | Win Rate [%]: {win*100 / trades:.2f}")
print(f"Profit[$]: {profit:.2f}")
display(df)


In [ ]:
# FILTER - SYMBOLS TO TRADE
blacklist = df[(df['SQN'] < 1) | (df['Equity Final [$]'] == 100)]
blacklist = blacklist.index.to_list() + ['USDCUSDT']

symbols_to_trade = [item for item in get_tickers_usdt() if item not in blacklist]

print(f'Symbols to trade: {len(symbols_to_trade)}')
print(symbols_to_trade)

In [ ]:
# LEVERAGE SETUP
from binance_futures import startup_trade

LEVERAGE = 10
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(startup_trade, symbol, 'ISOLATED', LEVERAGE): symbol for symbol in symbols_to_trade}
    for future in concurrent.futures.as_completed(futures):
        symbol = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"Error - {symbol}: {e}")